### ESM-1: two generators, single load, single bus optimization problem.

In [83]:
### Problem formulation
# generator 1: "gas" (CCGT), marginal cost 70 EUR/MWh, capacity 50 MW
# generator 2: "coal" (hard coal), marginal cost 40 EUR/MWh, capacity 100 MW
# load: "UA" (Ukraine), 120 MW
# single time step

In [84]:
import pypsa
import numpy as np
import pandas as pd
import linopy

### Create PyPSA network with components of the problem

In [85]:
n = pypsa.Network()

In [86]:
n

Empty PyPSA Network
Components: none
Snapshots: 1

In [87]:
n.add("Bus", "UA", v_nom=380)

In [88]:
n.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
UA,380.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,,


In [89]:
n.add(
    "Generator",
    "gas",
    bus="UA",
    p_nom_extendable=False,
    marginal_cost=70,  # €/MWh
    p_nom=50,  # MW
)
n.add(
    "Generator",
    "coal",
    bus="UA",
    p_nom_extendable=False,
    marginal_cost=40,  # €/MWh
    p_nom=100,  # MW
)

In [90]:
n.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
gas,UA,PQ,,50.0,False,0.0,inf,0.0,1.0,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
coal,UA,PQ,,100.0,False,0.0,inf,0.0,1.0,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [91]:
# add load
n.add(
    "Load",
    "Ukraine",
    bus="UA",
    p_set=pd.Series([120], index=["now"]),  # MW
)

In [92]:
n.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
Ukraine,UA,,,0.0,0.0,-1.0


### Solve with PyPSA optimize module (use the default mathematical problem)

In [93]:
n.optimize(solver_name="glpk")

INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 5.40e+03
Solver model: not available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-ffk61oqh.lp --output /tmp/linopy-solve-jqt6ue6_.sol
Reading problem data from '/tmp/linopy-problem-ffk61oqh.lp'...
5 rows, 2 columns, 6 non-zeros
35 lines were read
GLPK Simplex Optimizer 5.0
5 rows, 2 columns, 6 non-zeros
Preprocessing...
~     0: obj =   5.400000000e+03  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/linopy-solve-jqt6ue6_.sol'...


('ok', 'optimal')

In [94]:
n.objective

5400.0

In [95]:
n.generators_t.p

Generator,gas,coal
snapshot,,
now,20.0,100.0


### Explore pypsa model

In [96]:
n.optimize.create_model()

Linopy LP model

Variables:
----------
 * Generator-p (snapshot, Generator)

Constraints:
------------
 * Generator-fix-p-lower (snapshot, Generator-fix)
 * Generator-fix-p-upper (snapshot, Generator-fix)
 * Bus-nodal_balance (Bus, snapshot)

Status:
-------
initialized

In [97]:
n.model.objective

Objective:
----------
LinearExpression: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]
Sense: min
Value: None

In [98]:
n.model.constraints

linopy.model.Constraints
------------------------
 * Generator-fix-p-lower (snapshot, Generator-fix)
 * Generator-fix-p-upper (snapshot, Generator-fix)
 * Bus-nodal_balance (Bus, snapshot)

In [99]:
n.model.constraints["Generator-fix-p-upper"]

Constraint `Generator-fix-p-upper` (snapshot: 1, Generator-fix: 2):
-------------------------------------------------------------------
[now, gas]: +1 Generator-p[now, gas]   ≤ 50.0
[now, coal]: +1 Generator-p[now, coal] ≤ 100.0

In [100]:
n.model.constraints["Bus-nodal_balance"]

Constraint `Bus-nodal_balance` (snapshot: 1, Bus: 1):
-----------------------------------------------------
[now, UA]: +1 Generator-p[now, gas] + 1 Generator-p[now, coal] = 120.0

### Let's write optimization problem manually (reproduce PyPSA model for our problem)

In [101]:
# remove all constraints
for name in list(n.model.constraints):
    n.model.remove_constraints(name)

In [102]:
n.model.constraints

linopy.model.Constraints
------------------------
<empty>

In [103]:
# remove objective
n.model.objective.expression = linopy.LinearExpression(None, model=n.model)

In [104]:
n.model.objective.expression

LinearExpression
----------------
+0

In [105]:
# objective expression
(
    n.model["Generator-p"].loc[:, "gas"] * n.generators.marginal_cost.loc["gas"]
    + n.model["Generator-p"].loc[:, "coal"] * n.generators.marginal_cost.loc["coal"]
)

LinearExpression (snapshot: 1):
-------------------------------
[now]: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]

In [106]:
# set objective expression to the problem
n.model.objective.expression = (
    n.model["Generator-p"].loc[:, "gas"] * n.generators.marginal_cost.loc["gas"]
    + n.model["Generator-p"].loc[:, "coal"] * n.generators.marginal_cost.loc["coal"]
)

In [107]:
# check it is there
n.model.objective

Objective:
----------
LinearExpression: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]
Sense: min
Value: None

In [108]:
n.model.add_constraints(
    n.model["Generator-p"].sum(dims="Generator") == n.loads_t.p_set["Ukraine"],
    name="nodal_balance",
)

Constraint `nodal_balance` (snapshot: 1):
-----------------------------------------
[now]: +1 Generator-p[now, gas] + 1 Generator-p[now, coal] = 120.0

In [109]:
n.model.add_constraints(n.model["Generator-p"].loc[:, "gas"] >= 0, name="p_lower_gas")

Constraint `p_lower_gas` (snapshot: 1):
---------------------------------------
[now]: +1 Generator-p[now, gas] ≥ -0.0

In [110]:
n.model.add_constraints(n.model["Generator-p"].loc[:, "coal"] >= 0, name="p_lower_coal")

Constraint `p_lower_coal` (snapshot: 1):
----------------------------------------
[now]: +1 Generator-p[now, coal] ≥ -0.0

In [111]:
n.model.add_constraints(
    n.model["Generator-p"].loc[:, "gas"] <= n.generators.p_nom.loc["gas"],
    name="p_upper_gas",
)

Constraint `p_upper_gas` (snapshot: 1):
---------------------------------------
[now]: +1 Generator-p[now, gas] ≤ 50.0

In [112]:
n.model.add_constraints(
    n.model["Generator-p"].loc[:, "coal"] <= n.generators.p_nom.loc["coal"],
    name="p_upper_coal",
)

Constraint `p_upper_coal` (snapshot: 1):
----------------------------------------
[now]: +1 Generator-p[now, coal] ≤ 100.0

In [113]:
# check that we did a good job
n.model.constraints

linopy.model.Constraints
------------------------
 * nodal_balance (snapshot)
 * p_lower_gas (Generator, snapshot)
 * p_lower_coal (Generator, snapshot)
 * p_upper_gas (Generator, snapshot)
 * p_upper_coal (Generator, snapshot)

In [114]:
n.model.solve(solver_name="glpk")

INFO:linopy.model: Solve problem using Glpk solver


INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 5.40e+03
Solver model: not available
Solver message: optimal



GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-j6fpuhj5.lp --output /tmp/linopy-solve-jc53ba62.sol
Reading problem data from '/tmp/linopy-problem-j6fpuhj5.lp'...
5 rows, 2 columns, 6 non-zeros
33 lines were read
GLPK Simplex Optimizer 5.0
5 rows, 2 columns, 6 non-zeros
Preprocessing...
~     0: obj =   5.400000000e+03  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/linopy-solve-jc53ba62.sol'...


('ok', 'optimal')

In [115]:
n.model.objective

Objective:
----------
LinearExpression: +70 Generator-p[now, gas] + 40 Generator-p[now, coal]
Sense: min
Value: 5400.0

In [116]:
n.generators_t.p

Generator,gas,coal
snapshot,,
now,20.0,100.0
